In [1]:
# Import required libraries
import pandas as pd
import jupyter_dash
from dash import html, dcc, Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv", index_col='Unnamed: 0')
spacex_df['counter'] = 1
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = jupyter_dash.JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                    ],
                                    value='ALL',
                                    placeholder='Select a launch site',
                                    searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
)
def get_pie(selected_site):
    if selected_site=='ALL':
        fig = px.pie(
            spacex_df[spacex_df['class']==1],
            values='class',
            names='Launch Site',
            title='Number of Successful Launches by Site'
        )
    else:
        fig = px.pie(
            spacex_df[spacex_df['Launch Site']==selected_site],
            values='counter',
            names='class',
            color_discrete_sequence=['Red', 'Green'],
            title=('Successful and Failed Launches for Site ' + selected_site)
        )
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id="payload-slider", component_property="value")]
)
def get_scatter(selected_site, payload):
    # filter the dataframe based on the chosen site
    if selected_site=='ALL':
        filtered_df = spacex_df.copy()
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==selected_site]
    # further filter based on the payload slider
    filtered_df = filtered_df[
        (filtered_df['Payload Mass (kg)'] >= payload[0]) &
        (filtered_df['Payload Mass (kg)'] <= payload[1])
    ]
    
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title='Launches by Payload Mass and Booster Version'
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash app running on http://127.0.0.1:8050/
